<div style='width=100%; display:flex;flex-direction:row'><img  src=https://universidadeuropea.com/resources/media/images/universidad-europea-logo_poc9mEM.original.png width=100  style='  margin-left: auto;margin-right: auto; width: 25%; height:25%;'><img  src=https://i.ibb.co/1068C7j/EATEASER.jpg width=100 style='  margin-left: auto;margin-right: auto; width: 10%;height:25%;'></div>

<div style='margin:auto;text-align: center;font-family: "Times New Roman", Times, serif; font-weight: bold;'>PROYECTO COMUTACIONAL<br><br>EATEASER - VOZ A TEXTO</div>

<div style='width:100%; display:flex;flex-direction:row'>
    <div style='width:50%;margin-right:5cm;'>
        <p style='font-family: "Times New Roman", Times, serif; font-weight: bold;'>ESTUDIANTES</p>
<ul style='font-family: "Times New Roman", Times, serif;'>
    <li>Adilem Dobras 21911633</li><li>Roberto Echevarria 21823680</li><li>Carlos Gonzales 22067726</li><li>Juan Carlos Rondeau 21816176</li></ul> </div>

</div>

<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">1. Importamos las librerias</h2>

In [1]:
import glob
import os
import re
import requests
import pathlib
import sys

try:
    from pytube import YouTube
    from pytube import Playlist
except ModuleNotFoundError:
    !pip install pytube
try:
    import speech_recognition as sr
except ModuleNotFoundError:
    !pip install SpeechRecognition
try:
    from pydub import AudioSegment
    from pydub.silence import split_on_silence
except:
    !pip install pydub
try:
    import moviepy.editor as mp
except:
    !pip install moviepy
try:
    from bs4 import BeautifulSoup
except:
    !pip install beautifulsoup4
try:
    from nltk.stem import PorterStemmer
    from nltk.tokenize import word_tokenize
except:
    !pip install nltk  

C:\Users\thena\.conda\envs\EatEaser\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">2. Inicio del programa</h2>

<h3  style='font-family: "Times New Roman", Times, serif; font-weight: bold;text-align:center;font-size:14px'>CLASE CONTROLADORVIDEO</h3><p style='font-family: "Times New Roman", Times, serif; font-size:14px'>En esta clase se realizará los ajustes para manejar el video recibido y manipularlo.</p>


In [2]:
class ControladorVideo:
    def __init__(self,enlace): 
        self._idvideo = self.reenumerar()
        self.enlacevideo=enlace
        self.yt=YouTube(self.enlacevideo)
        self.nombrevideo=''
        self.titulovideo=self.yt.title
        self.autorvideo=self.yt.author
        self.fechavideo=self.yt.publish_date
        self.duracionvideo=self.yt.length
    """|DESCARGAR VIDEO URL: descarga el video de youtube
       |return: devuelve una ruta absoluta"""
    def descargarVideoURL(self):
        recetasVideos = 'recetasvideos/'
        #aqui creo un nuevo id para el nuevo video
        self._idvideo= self._idvideo+1
        #esta sera el archivo del video y su nuevo nombre
        nombre='receta'+str(self._idvideo)
        #le pedimos al pytube que solo nos descargue el audio y lo descargamos
        t=self.yt.streams.filter(file_extension='mp4').first().download(output_path=recetasVideos,filename=nombre+'.mp4')
        #devolvemos el nombre
        return nombre
    """|PARSEO VIDEO: pasa el video de .mp4 a .wav
       |nombre: es un string que se colocara el nombre del video
       |return: devuelve el nuevo nombre del audio en .wav"""
    def parseoVideo(self,nombre):
        recetasVideos = 'recetasvideos/'
        #tomamos el video en mp4 
        track = mp.VideoFileClip(recetasVideos+nombre+'.mp4')
        #cambiamos el video a .wav
        nombre_wav="{}.wav".format(nombre)
        track.audio.write_audiofile(recetasVideos+nombre_wav)
        track.close()
        return nombre
    """|REENUMERAR: Devuelve cual es el id del siguiente video.
       |primer for: recorre cada carpeta de /recetastextos
       |segundo for: cuenta cada archivo dentro de /recetastextos/* """
    def reenumerar(self):
        contador = 0
        for item in pathlib.Path("./recetastextos/").iterdir():
            if item.is_file():
                contador += 1
            elif item.is_dir():
                for subitem in item.iterdir():
                    if subitem.is_file():                     
                        contador += 1
        return contador
    """|SPEECH TEXT:Transforma el audio a texto
       |nombre: es un string que se colocara el nombre del video
       |return: devuelve un string con el texto devuelto"""
    def speech_text(self,nombre):
        recetasVideos = 'recetasvideos/'
        #instanciamos el recognizer
        r = sr.Recognizer()
        audio = sr.AudioFile(recetasVideos+nombre)
        with audio as source:
            audio_file = r.record(source)
        #transcribimos el audio a texto
        result = r.recognize_google(audio_file, language = 'es-ES')
        return result
        

<h3  style='font-family: "Times New Roman", Times, serif; font-weight: bold;text-align:center;font-size:14px'>CLASE DEPURADOR</h3><p style='font-family: "Times New Roman", Times, serif; font-size:14px'>En esta clase se realizará el proceso de extraccion, transformacion y carga de nuestro programa EATEASER.</p>


In [3]:
#si el video es mayor de 3 minutos no funciona
#si el video es en ingles no funciona
class Depurador:
    def __init__(self): 
        self.rec=RecursosAdicionales()
    """|VIDEO: proceso etl donde extraemos al informacion del video 
       |enlace: es un string que se colocara el enlace del video"""
    def video(self,enlace):
        try:
            #instanciamos el controlador de videos
            cv=ControladorVideo(enlace)
            #paso 1: descargamos el video
            cv.nombrevideo=cv.descargarVideoURL()
            #paso 2: pasamos el video a .wav
            nombre=cv.parseoVideo(cv.nombrevideo)
            #paso 3: pasamos el .wav a texto
            try:                
                num_segm=self.rec.segcionarXsilencios(nombre)
                result=""
                for i in range(num_segm):
                    result=result+str(cv.speech_text("../temp_audios/{}_extracto{}.wav".format(nombre,i+1)))
                    result=result+" "
                #paso 4: escribimos el texto recibido en un txt
                self.rec.eliminacionWav_Audios("temp_audios")
                try:
                    quitarEmojis = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 'NULL')
                    tituloSinEmojis=cv.titulovideo.translate(quitarEmojis)
                    autorSinEmojis=cv.autorvideo.translate(quitarEmojis)
                    resultado=self.rec.escritura(cv.nombrevideo,"Titulo:"+tituloSinEmojis+"\n"+"Autor:"+autorSinEmojis+"\n"+"Fecha Publicacion:"+str(cv.fechavideo)+"\n"+"Entradilla:"+result)
                    self.rec.eliminacionMp4()
                except Exception as e:
                    print(repr(e))
                    print("No se ha podido eliminar los caracteres corruptos el video: "+ cv.nombrevideo + " - "+ cv.titulovideo)
                    #paso 5: eliminamos los mp4
                    self.rec.eliminacionMp4()
                    return resultado   
            except Exception as e:
                print(repr(e))
                print("No se ha podido transcribir el video: "+ cv.nombrevideo + " - "+ cv.titulovideo+" - "+cv.enlacevideo)
                self.rec.eliminacionMp4()
                self.rec.eliminacionWav_Audios("temp_audios")

                return None
        except Exception as e:
            print(repr(e))
            print("No se ha podido descargar el video: "+ cv.nombrevideo + " - "+ cv.titulovideo)
            return None
    def lista(self, enlace):
        playlist_urls = Playlist(enlace)
        for url in playlist_urls:
            self.video(url)
    def transformacion(self):
        print()
    def carga(self):
        print()

In [4]:
#aqui iran lecturas-escrituras-guardar-eliminar cosas en bases de datos
class RecursosAdicionales:
    """|ESCRITURA: escribe textos txt
       |nombre: nombre del 
       |return: devuelve el audio en texto"""
    #yo siento que en vez de cambiar de carpeta los textos...deberiamos eliminar los mp4
    #y si lo mas factible aqui es que guardemos el video por su nombre?
    #creo que no debemos contar los files porque cansan el programa
    
    def escritura(self,nombre,texto):
        recetasTextos = './recetastextos/'
        if not(os.path.exists(recetasTextos)):
            os.mkdir(recetasTextos)
        f = open(recetasTextos+nombre+'.txt', 'w')
        f.write(texto)
        f = open(recetasTextos+nombre+'.txt', "r")
        print(f.read())
        f.close()
    def eliminacionMp4(self):
        recetasVideos = './recetasvideos/'
        py_files = glob.glob(recetasVideos+'*.mp4')
        for py_file in py_files:
            try:
                os.remove(py_file)
            except OSError as e:
                print(f"Error:{ e.strerror}")
                
    def eliminacionWav_Audios(self, path):
        url = './'+path+'/'
        py_files = glob.glob(url+'*.wav')
        for py_file in py_files:
            try:
                os.remove(py_file)
            except OSError as e:
                print(f"Error:{ e.strerror}") 
    
    def segcionarXsilencios(self,audio):
        import numpy as np
        from pydub import AudioSegment as aseg

        audio1=aseg.from_wav("./recetasvideos/"+audio+".wav")
        # this is an array
        samples = audio1.get_array_of_samples()
        segundo=88521
        index=[]
        for i in range(0,len(samples),int(segundo/5)):
            dataSeg = samples[i:int(segundo/5)+i]
            media=np.mean(dataSeg)
            var=np.var(dataSeg)
            if -10<=media<=10 and var<=1900:
                index.append(i)

        borrar=[]
        guardado=0
        for i in range(len(index)-1):
            if index[i+1]<=index[i]+(20*segundo):
                if i==0:
                    tiempo=(index[i])/segundo
                else:
                    tiempo=(index[i+1]-guardado)/segundo
                if tiempo<=120:
                    borrar.append(i)
                else:
                    print("guardados: " + str(index[i]/segundo))
                    guardado=index[i]
            else:
                guardado=index[i]

        final=np.delete(index, borrar, axis=0) 
        print("------------------------------------------------")
        print(final)
        print("------------------------------------------------")
        extractos=[]
        for i in range(len(final)):
            if i==0:
                extractos.append(samples[:final[i]])
            else:
                extractos.append(samples[final[i-1]:final[i]])
        for i in range(len(extractos)):
            nombre=""
            new_sound = audio1._spawn(extractos[i])
            nombre="temp_audios/{}_extracto{}.wav".format(audio,i+1)
            new_sound.export(nombre,format="wav")
        return len(final)

In [5]:
class WebScrap:
    def __init__(self): 
        self.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    def request(self, url):
        request1 = requests.get(url, headers=self.headers)
        html = request1.content
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    def verificar_alimento(self,alimento):
        soup = self.request( 'https://www.themealdb.com/api/json/v1/1/search.php?s='+alimento)
        print(soup)
        

<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">3. Main</h2>

In [6]:
#y si tambien vemos si le permitimos al usuario que meta videos?
dep=Depurador()
if __name__ == '__main__':
    #dep.video('https://www.youtube.com/watch?v=6PzQY1E2s2g&list=PLxHmjpcgU5ArC2rY5cpoIcZoVKB_0UHfR&ab_channel=PlatosF%C3%A1cilesconTamara')
    #dep.video('https://www.youtube.com/watch?v=PsqR5M8rdjA&list=LL&index=9&t=4s')
    #dep.video('https://www.youtube.com/watch?v=xfYcM_jHgPY')
    #dep.video('https://www.youtube.com/watch?v=wiCfqc5W-yo')
    #error_nuevo#dep.video('https://www.youtube.com/watch?v=3DnPkf9rP_0')
    #error_nuevo#dep.video('https://www.youtube.com/watch?v=xVsgKMZFCZY')
    #dep.video('https://www.youtube.com/watch?v=rpCe0RPMY94')
    #dep.video('https://www.youtube.com/watch?v=rv4gLMa-FYk')
    #dep.video('https://www.youtube.com/watch?v=VS8zYxBj4r8')
    #dep.video('https://www.youtube.com/watch?v=o99JXrEkZoo')
    #dep.video('https://www.youtube.com/watch?v=lKkg5L23b3M')
    #dep.video('https://www.youtube.com/watch?v=PsqR5M8rdjA&t=14s')
    #dep.video('https://www.youtube.com/watch?v=IvZaAL6qYe0&t=29s')
    #dep.video('https://www.youtube.com/watch?v=SIMQBuuyE9M')
    #dep.video('https://www.youtube.com/watch?v=_YoZfg7R8Hk')
    #dep.video('https://www.youtube.com/watch?v=Zv7KdlOBk7Y')
    #dep.video('https://www.youtube.com/watch?v=mFcN4btaZyI&t=2s')
    #dep.video('https://www.youtube.com/watch?v=sRmmQBBln9Q')
    #dep.video('https://www.youtube.com/watch?v=-QoTJJJfeEE')
    #dep.video('https://www.youtube.com/watch?v=JRY5obPKPzo&list=PLxHmjpcgU5ArC2rY5cpoIcZoVKB_0UHfR&index=5&ab_channel=PlatosF%C3%A1cilesconTamara')
    #dep.video('https://www.youtube.com/watch?v=stFmx7OCy1k&ab_channel=RecetasdeEsbieta')
    #error_videomuylargo#dep.video("https://www.youtube.com/watch?v=qqTqePGIjhc")
    #dep.lista("https://youtube.com/playlist?list=PLQwfLPYiFlOsS9x6zgeZmFRLqDx3poZvw")
    dep.lista("https://www.youtube.com/playlist?list=PLIsSIvqffHZvM2v1QS5Zi0MUL258EKLPq")
    

MoviePy - Writing audio in recetasvideos/receta24.wav


MoviePy - Done.
------------------------------------------------
[ 548824 9276896]
------------------------------------------------
Titulo:Churrasco a la plancha con sorpresa / Carne a la plancha - Paulina Cocina
Autor:Paulina Cocina
Fecha Publicacion:2015-06-03 00:00:00
Entradilla:la gente un poquito dice la plancha una forma de alegrar un vicio a la plancha también un poco rara pero qué buenísimo confía en mí esta noche para saber más así que está pido esta pasta pero me vas a mandar un mensaje puedes hacer la Paulina gracias gracias gracias nunca más vuelvo a desconfiar en voz presta atención TSP entonces qué vas a comer todavía no están las planchas recaliente como si tienes un momento le vas a poner el pollito de lado de la mesa de café mientras está haciendo le pones por arriba y eso no es el tiempo que dejes tu churrasco un poco jugado todo entonces lo estoy dejando un minuto fuerte y luego bajo el fuego medio otro minuto y lo vemos prontito de cada lado cuenta y lo terminamos d

MoviePy - Done.
------------------------------------------------
[  495712  7258640 12162648]
------------------------------------------------
Titulo:Cómo hacer tortitas ¡RÁPIDO!: Tortitas de maíz / Choclo  - Paulina Cocina
Autor:Paulina Cocina
Fecha Publicacion:2015-07-01 00:00:00
Entradilla:vamos a hacer tortitas de coco tortilla de choclo no de coco particularidad que se pueden hacer dulces o saladas quieres una pasta ING agregamos el huevo y pequeñito la mezcla bien bien yo cereales salpimienta y el queso racha si son las dulces en esta parte le pones el azúcar y el queso que se rifa si quieres vas a poner chorrito de aceite en una sartén y no me gusta escuchar más a empezar a poner la mezcla de las tortitas a todo medio a maravisa así se van haciendo las tortitas cuando las ves doraditas de un lado las das vueltas ya y las vas pasando importante hacerlo fuego medio a fuego fuerte se van a quemar a fuego suave se van a acercar las ponemos aquí en una fuentecita en mi caso le tiramo

MoviePy - Done.
guardados: 127.99855401543137
------------------------------------------------
[  318672 11330560 12941624 16960432]
------------------------------------------------
UnknownValueError()
No se ha podido transcribir el video: receta26 - Receta de Sopa Paraguaya - Paulina Cocina T04 #10 - https://www.youtube.com/watch?v=XMur7aWyCH8
MoviePy - Writing audio in recetasvideos/receta26.wav


MoviePy - Done.
------------------------------------------------
[ 336376 5470536 9666384]
------------------------------------------------
Titulo:Pastel de Verduras (Strata): absolutamente GENIAL- Paulina Cocina
Autor:Paulina Cocina
Fecha Publicacion:2015-04-15 00:00:00
Entradilla:estoy firmando hacer una receta de verduras que me robó el corazón nunca nunca es fácil de verdura se trata de verduras le van cantar señora y si lo Pokémon y familia sonriendo a la cámara es típico como si fuera una tarta pero como quieras es tonto te van a cantar en la señora ingredientes para una fuente mediana son un paquete de espinaca una cebolla cuatro o cinco tomates secos unos 3 pancitos como los que te voy a mostrar o el equivalente en volumen de los ciegos tenga tres huevos sal pimienta y un chorrito de aceite guarnición para usar como plato único final Cataluña y los tomates secos que los tengo hidratando hace un ratito en aguas tibias siguiente que hacemos es en 1 1 8 vamos a poner un chorrito d

...

MoviePy - Done.
guardados: 1261.5857480145953
------------------------------------------------
[   371784 111676832 111783056]
------------------------------------------------
RequestError('recognition request failed: Bad Request')
No se ha podido transcribir el video: receta52 - COCINÁ 1 DÍA PARA COMER SALUDABLE TODA LA SEMANA - El #1000prep VUELACABEZAS🤯 - https://www.youtube.com/watch?v=8QHjLvfg8aw
MoviePy - Writing audio in recetasvideos/receta52.wav


MoviePy - Done.
------------------------------------------------
[56829840]
------------------------------------------------
RequestError('recognition request failed: Bad Request')
No se ha podido transcribir el video: receta52 - 21 ENSALADAS que NO SON lechuga y tomate: la BIBLIA de la ensalada - Paulina Tirapostas E00 - https://www.youtube.com/watch?v=G3JfV4BCuwg
MoviePy - Writing audio in recetasvideos/receta52.wav


MoviePy - Done.
guardados: 164.998136035517
------------------------------------------------
[       0 14605800 15154624]
------------------------------------------------
UnknownValueError()
No se ha podido transcribir el video: receta52 - POSEIDÓN | Destripando la Historia | Canción - https://www.youtube.com/watch?v=2-VTUyI3w7U
MoviePy - Writing audio in recetasvideos/receta52.wav


MoviePy - Done.
------------------------------------------------
[55926936]
------------------------------------------------
RequestError('recognition request failed: Bad Request')
No se ha podido transcribir el video: receta52 - 7 IDEAS para HALLOWEEN 🎃 - 7 Comidas de halloween que funcionan, 1 que NO funciona - https://www.youtube.com/watch?v=wEgl6I1If3M
MoviePy - Writing audio in recetasvideos/receta52.wav


MoviePy - Done.
guardados: 763.5913737983077
------------------------------------------------
[ 6656704  9914240 22041480 32203576 35726672 39426808 67593872 67629280]
------------------------------------------------
RequestError('recognition request failed: Bad Request')
No se ha podido transcribir el video: receta52 - LA MEJOR TARTA DE BERENJENAS QUE PROBARON 🍆- Receta de mi tía Alicia - https://www.youtube.com/watch?v=riVDxV-sXP0
